# Reverse Geocoding using HERE API

Given a csv with Geographic Coordinates, extracts full address (incluiding postal code)

More information: https://developer.here.com/api-explorer/rest/geocoder/reverse-geocode

In [17]:
import pandas
import numpy
import requests
import json
import time
import os
from io import StringIO

In [18]:
temp = os.path.expanduser('C:\\Users\\pmeji\\Dropbox\\CIE\\ProyectoENLACE\\Mapas\\Build\\temp')
output = os.path.expanduser('C:\\Users\\pmeji\\Dropbox\\CIE\\ProyectoENLACE\\Mapas\\Build\\output')
source =  os.path.expanduser('C:\\Users\\pmeji\\Dropbox\\CIE\\ProyectoENLACE\\Mapas\\Build\\source')

In [19]:
filename = 'cct_lat_lon.csv'
addr= os.path.join(source, filename)
df = pandas.read_csv(addr, encoding = 'unicode_escape')

In [20]:
df.head()

,cct,longitud,latitud
0,25KJN0346R,-108.27167,26.779722
1,25KJN0347Q,-108.08434,26.491606
2,25KJN0348P,-108.35389,26.574444
3,25KJN0350D,-108.18266,26.407717
4,25KJN0351C,-106.78722,24.259167


In [21]:
len(df)

499181

### Data cleaning

Deletes empty observations and obersvations with non-existing coordinates

In [23]:
df_coor = df
print(len(df_coor))
df_coor = df_coor.dropna()
print(len(df_coor))

df_coor = df_coor[pandas.to_numeric(df_coor['longitud'], errors='coerce').notnull()]
df_coor = df_coor[pandas.to_numeric(df_coor['latitud'], errors='coerce').notnull()]


df_coor = df_coor.loc[(df_coor['latitud']< 90) & (df_coor['latitud']> 0)]
df_coor = df_coor.loc[(df_coor['longitud']< 180) & (df_coor['longitud']> -180)]
df_coor = df_coor.loc[(df_coor['longitud']!= 0) | (df_coor['latitud']!= 0)]


df_coor = df_coor.reset_index(drop=True)
len(df_coor)

499181
484922


472365

### Functions for reverse geocoding

In [ ]:
def flatten_json(y):
    out = {}
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x
    flatten(y)
    return out


#Given a range, extracts the address from that range and 
def reverseGeo(df_coor, mini, maxi, name):
    resDF = pandas.DataFrame()
    url_base    = 'https://reverse.geocoder.api.here.com/6.2/reversegeocode.json?prox=';
    extra = '&mode=retrieveAddresses&maxresults=1&gen=9&app_id=4PvhgylLABytx6S2AeoS&app_code=BVuMZxkU2w3K8DhtUUilFg' #This key no longer works. Go online and register to get a new one

    for i in range(mini,maxi):

        lon = df_coor['longitud'][i]
        lat = df_coor['latitud'][i]

        if (-90 < lat < 90 ) & (-180 < lon < 180):
            prox ='' + str(lat) + '%2C' + str(lon) + '%2C250'
            url = url_base + prox + extra #Builds url to make request
            
            try:
                f = requests.get(url) 
                if (f.status_code == 200):
                    json_f  = json.loads(f.text)
                    if (len(json_f["Response"]['View'])!=0):
                        info_cct = json_f["Response"]["View"][0]['Result'][0]['Location']['Address']
                        flat_json  = flatten_json(info_cct)

                        apiDF      = pandas.DataFrame.from_dict(flat_json, orient='index')
                        apiDF      = apiDF.transpose()
                        apiDF['cct'] = df_coor['cct'][i]
                        print(df_coor['cct'][i],i)
                        resDF = resDF.append(apiDF, ignore_index = True)
            except requests.exceptions.RequestException as e:  # Handles exceptions
                print(e) 
        time.sleep(numpy.random.uniform())

    resDF.to_csv(file_out,encoding='utf-8-sig')
    return resDF

In [ ]:
size = len(df_coor)
step= 100000
df_final = pandas.DataFrame()

for i in range(0,size, step):
    mini = i
    maxi = i + step
    filename = '' + 'cct_dir_'+ str(mini) + '_' + str(maxi) + '.csv'
    file_out = os.path.join(temp, filename)
    df_1 = reverseGeo(df_coor, mini, maxi, file_out)
    df_final.append(df_1,  ignore_index = True)

name = os.path.join(temp, "cct_dir_0_"+size)    
df_final.to_csv(name,encoding='utf-8-sig')  